# Download and generate from HF

In [ ]:
!pip install -U transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 136.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "Dushyant4342/ft-llama3-8b-credit-analyst" # 16 GB 8.32 B params (32M fine tuned params)

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

In [ ]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [ ]:

# 1. Define the expert system prompt
#system_prompt = """You are an expert credit analyst. Your role is to analyze a customer's credit data and generate a highly concise summary of the most important positive and negative changes. You must adhere to the length and format constraints given by the user."""


system_prompt = """You are a helpful and insightful financial advisor. Your primary role is to answer user questions about their credit profile. You will be given a user's question followed by their detailed credit report data.

You MUST use the provided data to formulate your answer. Do NOT simply summarize the data. Instead, analyze it to provide specific, actionable advice that directly answers the user's question. Keep your responses concise and clear."""


# 2. Provide the customer data (as a structured string)
customer_data = """

--- Credit Profile Report for Customer: 3510646997537260646 ---

## Key Metric Summary
-  Risk Score : 739.0 (was 719.0)
-  Overall Utilization : 2.15% (was 0.00%)
-  Credit Card Utilization : 18.89% (was 43.98%)
-  Total Active Accounts : 2.0 (was 1.0)

## Current Credit Mix
-  Total Accounts : 2 (2.0 active)
-  Active Secured Products : 1
-  Active Unsecured Products : 1
-  Active Lender Distribution : Public(0), Private(1), NBFC(0), Corporate(0), Foreign(1)

## Payment Behavior History
-  Past Delinquencies : User has been delinquent on 0 accounts in their history.

## Account Details Breakdown

-  Account : SBMBKINDIA -  Secured Credit Card (#J6V5)
  -  Status : New Account (Active)
  -  DPD : 0.0 days (was 0.0 days)
  -  Utilization : 18.89% (was 0.00%)
  -  Info Change : Account type is now '31.0' (was '0.0')

-  Account : IDFC FIRST BANK -  Consumer Loan (#8927)
  -  Status : Active
  -  DPD : 0.0 days (was 0.0 days)



"""


In [ ]:
# 3. Give a specific command (Examples)

user_command = "Start your summary with the credit score change, then list the main reasons for it."
#user_command = "Summarize the key takeaways in two bullet points one Good & other Bad."

#user_command = "Tell me the most important reason why my credit card changed in one line"
#user_command = "How can I improve my credit score"


# 4. Format the final prompt for the model
user_content = f"""
**User's Question:**
{user_command}

**Credit Profile Data:**
{customer_data}
"""

user_command # it is not being followed by the model

'Start your summary with the credit score change, then list the main reasons for it.'

In [ ]:

#user_content = f"{user_command}\n\n--- DATA ---\n{customer_data}"

msgs = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_content},
]

prompt = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#inputs

In [ ]:
#user_content

In [ ]:
%%time
# 5. Generate the response
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=250, #128
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")],
    )

response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
#print(response)
print(f"COMMAND: {user_command}\n\nMODEL RESPONSE:\n{response}")


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


COMMAND: Start your summary with the credit score change, then list the main reasons for it.

MODEL RESPONSE:
Good:- User's score has increased between 2 months by 20.0 points.
Bad:- User has opened new following accounts: SBMBKINDIA ( Secured Credit Card).
Good:- User's cc utilisation has decreased by 57.00 percentage points.
Good:- User's overall utilisation is 2.15% and cc utilisation is 18.89%.
Good:- User has utilisation less than 30% in the following accounts: SBMBKINDIA (19%), IDFC FIRST BANK (0%).
CPU times: user 7.64 s, sys: 359 ms, total: 8 s
Wall time: 9.01 s


In [ ]:
print(response)

Good:- User's score has increased between 2 months by 20.0 points.
Bad:- User has opened new following accounts: SBMBKINDIA ( Secured Credit Card).
Good:- User's cc utilisation has decreased by 57.00 percentage points.
Good:- User's overall utilisation is 2.15% and cc utilisation is 18.89%.
Good:- User has utilisation less than 30% in the following accounts: SBMBKINDIA (19%), IDFC FIRST BANK (0%).


# ROGUE SCORE

1. Quantitative Evaluation (Automatic Scores)
This method uses metrics to automatically compare the model's generated summary to a perfect, human-written "reference" summary.
Prepare a Test Set: You need a small set of customer_info examples (that the model has never seen) and a corresponding ideal summary for each, written by you.
Use ROUGE Score: The standard metric for summarization is ROUGE. It measures the overlap of words and phrases between the model's output and your reference summary. A higher ROUGE score generally means a better summary.
Run Evaluation: Use libraries like Hugging Face's evaluate to automatically calculate the ROUGE score across your entire test set.




2. The model's performance was validated on a holdout set of unseen customer profiles. Generated summaries were manually reviewed for factual accuracy, coherence, and their ability to correctly identify the most critical credit insights.



In [ ]:
labelled = """
Good:- User's score has increased between 2 months by 20.0 points.
Bad:- User has opened new following accounts: SBMBKINDIA ( Secured Credit Card).
Good:- User's cc utilisation has decreased by 57.05 percentage points.
Good:- User's overall utilisation is 2.15% and cc utilisation is 18.89%.
Good:- User has utilisation less than 30% in the following accounts: SBMBKINDIA (19%), IDFC FIRST BANK (0%)."""

In [ ]:
# 1. Install the necessary libraries
!pip install -q evaluate rouge_score
import evaluate
model_prediction = response
human_reference = labelled
rouge = evaluate.load('rouge')
results = rouge.compute(
    predictions=[model_prediction],
    references=[human_reference])

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00


## Score Range	Interpretation
0.60 and up	Excellent: The model's output is highly coherent and very similar to a human.

0.40 - 0.60	Good: The model is capturing the main points and is highly effective.

0.20 - 0.40	Fair: The model understands the basic topic but may miss key details.

Below 0.20	Poor: The summary has very little in common with the reference.

In [ ]:
print("\n--- Evaluation Results ---")
print(f"ROUGE-1: {results['rouge1']:.4f} (Measures overlap of single words)")
print(f"ROUGE-2: {results['rouge2']:.4f} (Measures overlap of word pairs)")
print(f"ROUGE-L: {results['rougeL']:.4f} (Measures longest common subsequence)")



--- Evaluation Results ---
ROUGE-1: 0.9851 (Measures overlap of single words)
ROUGE-2: 0.9697 (Measures overlap of word pairs)
ROUGE-L: 0.9851 (Measures longest common subsequence)


# Bert Score

BERTScore is a powerful metric because it compares the semantic meaning of the sentences using contextual embeddings, rather than just word overlap like ROUGE. This is great for judging the quality of abstractive summaries.



## How to Interpret the Scores
Precision: Checks if the words in your model's prediction are relevant and semantically similar to the reference.

Recall: Checks if your model's prediction captures all the key information present in the reference.

F1 Score: This is the harmonic mean of Precision and Recall and is usually the most important score to look at for overall quality.



In [ ]:
# 1. Install the necessary libraries
!pip install -q evaluate bert_score
import evaluate
# --- 2. Define your single prediction and reference ---
# The summary generated by your model
model_prediction = response
# The ideal, human-written summary for the same data
human_reference = labelled
# --- 3. Load and Calculate the BERTScore ---
# Load the BERTScore metric
bertscore = evaluate.load("bertscore")

# Compute the score by passing the strings inside lists
# We specify lang="en" to use an English-based model
results = bertscore.compute(
    predictions=[model_prediction],
    references=[human_reference],
    lang="en" # Use an English model for embeddings
)

# --- 4. Print the Results ---
# We look at the F1-score as the main indicator
print("--- BERTScore for a Single Response ---")
print(f"Precision: {results['precision'][0]:.4f}")
print(f"Recall:    {results['recall'][0]:.4f}")
print(f"F1 Score:  {results['f1'][0]:.4f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.8 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- BERTScore for a Single Response ---
Precision: 0.9989
Recall:    0.9984
F1 Score:  0.9987


In [ ]:
import torch
import gc
del bertscore
del results
gc.collect()

6388

In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

#rogue_score for multiple customers

In [ ]:
# --- 1. Installation ---
!pip install -q evaluate rouge_score sentencepiece

import torch
import pandas as pd
import evaluate  # The Hugging Face evaluation library
from tqdm import tqdm # A library to show progress bars
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- 2. Load Your Fine-Tuned Model ---
model_id = "Dushyant4342/ft-llama3-8b-credit-analyst"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()

# --- 3. Prepare Your Test Data ---
# In your real use case, you would load your holdout test data here.
# For this example, we'll create a small sample DataFrame.
test_data = {
    'customer_info': [
        """--- Credit Profile Report for Customer: 111111 ---
## Key Metric Summary
- Risk Score : 730 (was 755)
- Overall Utilization : 85.10% (was 60.25%)
- Total Active Accounts : 4 (was 3)
## Account Details Breakdown
- Account : SOME BANK - Credit Card (#1234)
  - Status : New Account (Active)
  - DPD : 35 days (was 0 days)""",
        """--- Credit Profile Report for Customer: 222222 ---
## Key Metric Summary
- Risk Score : 780 (was 770)
- Overall Utilization : 25.00% (was 55.00%)
- Total Active Accounts : 2 (was 3)
## Account Details Breakdown
- Account : ANOTHER BANK - Personal Loan (#5678)
  - Status : Closed this Period
  - DPD : 0 days (was 0 days)"""
    ],
    'customer_credit_update': [ # These are your "perfect" human-written answers
        "Bad:- User's score has reduced by 25 points due to a new 35-day delinquency on a new credit card account.",
        "Good:- User's score increased by 10 points after closing a personal loan and significantly reducing overall utilization."
    ]
}
test_df = pd.DataFrame(test_data)

# Get your two lists: predictions (empty for now) and references
predictions = []
references = test_df['customer_credit_update'].tolist()

# --- 4. Generate Predictions for the Test Set ---
system_prompt = "You are an expert credit analyst. Your role is to analyze a customer's credit data and generate a highly concise summary of the most important positive and negative changes."

print("Generating predictions on the test set...")
for index, row in tqdm(test_df.iterrows(), total=len(test_df)):
    customer_info = row['customer_info']

    # We ask for a summary, not a specific format, to test its core fine-tuned ability
    user_command = "Summarize the key updates."
    user_content = f"{user_command}\n\n--- DATA ---\n{customer_info}"

    msgs = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content},
    ]

    prompt = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=128, eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")])

    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    predictions.append(response)


# --- 5. Calculate the ROUGE Score ---
print("\nCalculating ROUGE scores...")
rouge = evaluate.load('rouge')

results = rouge.compute(predictions=predictions, references=references)

# Print the results
print("\n--- Evaluation Results ---")
print(f"ROUGE-1: {results['rouge1']:.4f} (Measures overlap of single words)")
print(f"ROUGE-2: {results['rouge2']:.4f} (Measures overlap of word pairs)")
print(f"ROUGE-L: {results['rougeL']:.4f} (Measures longest common subsequence)")